# nota bene

+ This notebook generates cooccurrence counts given Web of Science output.
+ No Web of Science output is included in this distribution.
+ This is **only** useful if you have Web of Science output you have collected yourself, and want to run analyses on that.

# Instructions

1. Edit the parameters to fit your needs
2. Run all cells

# Parameters

In [ ]:
basedir = "path/to/wos/data"
journal_keep = ["ETHNIC AND RACIAL STUDIES", "LAW & SOCIETY REVIEW", "DISCOURSE & SOCIETY", "SOCIOLOGICAL INQUIRY", "CONTRIBUTIONS TO INDIAN SOCIOLOGY", "SOCIETY & NATURAL RESOURCES", "RATIONALITY AND SOCIETY", "DEVIANT BEHAVIOR", "ACTA SOCIOLOGICA", "SOCIOLOGY-THE JOURNAL OF THE BRITISH SOCIOLOGICAL ASSOCIATION", "WORK EMPLOYMENT AND SOCIETY", "SOCIOLOGICAL METHODS & RESEARCH", "SOCIOLOGICAL PERSPECTIVES", "JOURNAL OF MARRIAGE AND FAMILY", "WORK AND OCCUPATIONS", "JOURNAL OF CONTEMPORARY ETHNOGRAPHY", "THEORY AND SOCIETY", "POLITICS & SOCIETY", "SOCIOLOGICAL SPECTRUM", "RACE & CLASS", "ANTHROZOOS", "LEISURE SCIENCES", "COMPARATIVE STUDIES IN SOCIETY AND HISTORY", "SOCIAL SCIENCE QUARTERLY", "MEDIA CULTURE & SOCIETY", "SOCIOLOGY OF HEALTH & ILLNESS", "SOCIOLOGIA RURALIS", "SOCIOLOGICAL REVIEW", "TEACHING SOCIOLOGY", "BRITISH JOURNAL OF SOCIOLOGY", "JOURNAL OF THE HISTORY OF SEXUALITY", "SOCIOLOGY OF EDUCATION", "SOCIAL NETWORKS", "ARMED FORCES & SOCIETY", "YOUTH & SOCIETY", "POPULATION AND DEVELOPMENT REVIEW", "SOCIETY", "JOURNAL OF HISTORICAL SOCIOLOGY", "HUMAN ECOLOGY", "INTERNATIONAL SOCIOLOGY", "SOCIAL FORCES", "EUROPEAN SOCIOLOGICAL REVIEW", "JOURNAL OF HEALTH AND SOCIAL BEHAVIOR", "SOCIOLOGICAL THEORY", "SOCIAL INDICATORS RESEARCH", "POETICS", "HUMAN STUDIES", "SOCIOLOGICAL FORUM", "AMERICAN SOCIOLOGICAL REVIEW", "SOCIOLOGY OF SPORT JOURNAL", "SOCIOLOGY OF RELIGION", "JOURNAL OF LAW AND SOCIETY", "GENDER & SOCIETY", "BRITISH JOURNAL OF SOCIOLOGY OF EDUCATION", "LANGUAGE IN SOCIETY", "AMERICAN JOURNAL OF ECONOMICS AND SOCIOLOGY", "ANNALS OF TOURISM RESEARCH", "SOCIAL PROBLEMS", "INTERNATIONAL JOURNAL OF INTERCULTURAL RELATIONS", "SOCIAL SCIENCE RESEARCH", "SYMBOLIC INTERACTION", "JOURNAL OF LEISURE RESEARCH", "ECONOMY AND SOCIETY", "INTERNATIONAL JOURNAL OF COMPARATIVE SOCIOLOGY", "SOCIAL COMPASS", "SOCIOLOGICAL QUARTERLY", "JOURNAL OF MATHEMATICAL SOCIOLOGY", "AMERICAN JOURNAL OF SOCIOLOGY", "REVIEW OF RELIGIOUS RESEARCH", "RURAL SOCIOLOGY", "JOURNAL FOR THE SCIENTIFIC STUDY OF RELIGION", "ARCHIVES EUROPEENNES DE SOCIOLOGIE", "CANADIAN JOURNAL OF SOCIOLOGY-CAHIERS CANADIENS DE SOCIOLOGIE", "POLISH SOCIOLOGICAL REVIEW"]
name_blacklist = [
    "*us", 'Press', 'Knopf', '(January', 'Co', 'London', 'Bros', 'Books', 'Wilson','[anonymous]'
]

# General Imports

In [1]:
%matplotlib inline

import sys; sys.path.append(_dh[0].split("citation-deaths")[0] + "citation-deaths")
from lib import *

from pathlib import Path
import re

In [ ]:
dcount = 0
total_inserts = 0
to_inserts = []
basedir = Path(basedir)

# keeps track of all the years seen for grouped citations
multi_year = defaultdict(lambda: defaultdict(int))

In [ ]:
# Instantiating counters
cnt_ind = defaultdict(lambda:defaultdict(int))
track_doc = defaultdict(lambda:defaultdict(set))
cnt_doc = defaultdict(lambda:defaultdict(int))

def cnt(term, space, doc):
    # it's a set, yo
    track_doc[space][term].add(doc)
    # update cnt_doc
    cnt_doc[space][term] = len(track_doc[space][term])
    # update ind count
    cnt_ind[space][term] += 1


In [ ]:
# This cell ensures there are not overflow errors while importing large CSVs

import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [ ]:
def fix_auths(auths):
    for a in auths:
        a = a.strip()
        if not len(a):
            continue

        a = a.split()[0].lower()
        yield a


def fix_refs(refs):
    for r in refs:
        yspl = re.split("((?:18|19|20)[0-9]{2})", r)

        if len(yspl) < 2:
            continue

        auth, year = yspl[:2]
        auth = auth.strip()
        year = int(year)

        if auth == "":
            continue

        auth = "&".join( fix_auths( auth.split(",") ) )

        yield ( auth, year, r )

In [1]:
book_groups = load_variable("groups.books")
article_groups = load_variable("groups.articles")

from collections import defaultdict

def get_reps(groups):
    ret = defaultdict(set)
    for k,v in groups.items():
        ret[v].add(k)
    ret = {
        k: list(v)[0]
        for k,v in ret.items()
    }
    return ret

book_group_reps = get_reps(book_groups)
article_group_reps = get_reps(article_groups)

NameError: name 'load_variable' is not defined

In [ ]:
for i, f in enumerate( list(basedir.glob("**/*.txt")) ):

    with f.open(encoding='utf8') as pfile:
        r = DictReader(pfile, delimiter="\t")
        rows = list(r)

    print("File %s/%s: %s" % (i, len(list(basedir.glob("**/*.txt"))),f))

    for i, r in enumerate(rows):

        if r['SO'] not in wjournals:
            continue

        if r['DT'] != "Article":
            continue

        refs = r["CR"].strip().split(";")
        refs = list( fix_refs(refs) )

        if not len(refs):
            continue

        #print(refs)

        dcount += 1
        if dcount % 10000 == 0:
            print("Document %s" % dcount)


        authors = r['AU'].split(";")
        authors = [x.strip() for x in authors]

        if False:
            for i in range(10):
                print("-"*20)


        uid = r['UT']


        try:
            int(r['PY'])
        except ValueError:
            continue



        for (auth,year,full_ref) in refs:
            if debug:
                print(full_ref)
            if 'DOI' not in full_ref and not len(re.findall(r', V[0-9]+, P[0-9]+', full_ref)):
                # splits off the author and year, and takes until the next comma
                full_ref = "|".join(
                    [auth]+
                    [x.strip().lower() for x in full_ref.split(",")[2:3]]
                )
            else:
                # just adds a fixed name and date to the front
                full_ref = "|".join(
                    [auth, str(year)] +
                    [",".join( full_ref.strip().lower().split(",")[2:] ).split(", doi")[0]]
                )
            if full_ref in book_groups:
                # retrieves retrospectively-computed groups
                full_ref = book_group_reps[
                    book_groups[full_ref]
                ]
                multi_year[full_ref].add(year)
            elif full_ref in article_groups:
                # retrieves retrospectively-computed groups
                full_ref = article_group_reps[
                    article_groups[full_ref]
                ]
            else:
                # a small minority, the ones which are dropped in this process anyways
                continue

            if debug:
                print(full_ref)
                print("--------------------")


            ref = (auth,year)

            cnt(r['SO'], 'fj', uid)
            cnt(int(r['PY']), 'fy', uid)
            cnt(ref[1], 'ty', uid)
            cnt((int(r['PY']), year), 'fy.ty', uid)
            cnt((r['SO'], year), 'fj.ty', uid)

            ref_str = "|".join( str(x) for x in ref )
            ref_str = full_ref.strip()
            #print(ref_str)

            if ref[0] in name_blacklist:
                continue
            if "*" in ref[0]:
                continue
                
            if r['SO'] not in journal_keep:
                continue

            cnt((r['SO'],int(r['PY'])), 'jy', uid)

            for a in authors:
                cnt(a, 'a', uid)
                cnt((a,int(r['PY'])), 'ay', uid)
                cnt((a,r['SO']), 'aj', uid)
                cnt((a,r['SO'], int(r['PY'])), 'ajy', uid)

                cnt((a,ref_str), 'ac', uid)

            cnt(ref_str, 'c', uid)
            cnt((ref_str, int(r['PY'])), 'cy', uid)
            cnt((ref_str, r['SO']), 'cj', uid)
            cnt((ref_str, r['SO'], int(r['PY'])), 'cjy', uid)

In [ ]:
# retrieve and use the MOST COMMON pub date for each
pubyears = {
    k:min(s) for k,s in multi_year.items()
    #if len(s)>1
}
varname = "pubyears.wos.all.uid"
save_variable(varname, pubyears)
print("saved %s" % varname)